# Use tool to enable LLM to take actions when needed

ref: https://platform.openai.com/docs/guides/function-calling?api-mode=responses

In [1]:
from llm_cookbook.utils.get_client import openai_client
from llm_cookbook.utils.tool_creation import create_tool, registry

%load_ext autoreload
%autoreload 2

In [16]:
import json

from pydantic import BaseModel, Field

In [6]:
# Tool parameter
class SearchStockPrice(BaseModel):
    name: str = Field(..., description="Search query for Stock name")


# Create tool function
@create_tool(
    name="search_stock_price",
    description="Search stock price from the database",
    parameters_model=SearchStockPrice,
)
def search_stock_price(name: str) -> str | int:
    """Implementation of stock price search functionality"""
    if name in ["AAPL", "TSM"]:
        return 100
    else:
        return f"Stock {name} is not in our database."


# Register the tool
registry.register(search_stock_price)

In [8]:
sys_prompt = "You are a helpful assistant that is able to privide info about stocks"
input_ = "what is the stock price of AAPL?"

message_stack = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": input_},
]


# Call the llm to get response
response1 = openai_client.chat.completions.create(
    messages=message_stack,
    model="gpt-4o",
    tools=[search_stock_price.to_openai_schema()],
)

In [11]:
# Finish reason
print(f"Finish reason: {response1.choices[0].finish_reason}")

Finish reason: tool_calls


In [14]:
# message concent is none
# but tool_calls contains info
response1.choices[0].message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_c7v6ix9i55XzyjjTq2gG2I8n', function=Function(arguments='{"name":"AAPL"}', name='search_stock_price'), type='function')])

In [24]:
for tool_call in response1.choices[0].message.tool_calls:
    # Add message into stack
    message_stack.append(response1.choices[0].message)

    # Get function name & parameters
    tool_name = tool_call.function.name
    tool_args = json.loads(tool_call.function.arguments)

    # execute the tool
    tool = registry.get_tool(tool_name)
    if tool:
        result = tool(**tool_args)
        print(f"Tool {tool_name} returned: {result}")

    # Add result from tool into stack
    message_stack.append(
        {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": str(result),
        }
    )

# Then call LLM again
response2 = openai_client.chat.completions.create(
    messages=message_stack,
    model="gpt-4o",
    tools=[search_stock_price.to_openai_schema()],
)

Tool search_stock_price returned: 100


In [25]:
# Finish reason
print(f"Finish reason: {response2.choices[0].finish_reason}")

response2.choices[0].message

Finish reason: stop


ChatCompletionMessage(content='The stock price of AAPL is $100.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

In [27]:
# check the message stack
message_stack

[{'role': 'system',
  'content': 'You are a helpful assistant that is able to privide info about stocks'},
 {'role': 'user', 'content': 'what is the stock price of AAPL?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_c7v6ix9i55XzyjjTq2gG2I8n', function=Function(arguments='{"name":"AAPL"}', name='search_stock_price'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_c7v6ix9i55XzyjjTq2gG2I8n',
  'content': '100'}]

In [29]:
# Try out one more example
sys_prompt = "You are a helpful assistant that is able to privide info about stocks"
input_ = "what is the captial of Greece"

message_stack = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": input_},
]


# Call the llm to get response
response1 = openai_client.chat.completions.create(
    messages=message_stack,
    model="gpt-4o",
    tools=[search_stock_price.to_openai_schema()],
)

# No need to run the tool
print(response1.choices[0].message.content)

The capital of Greece is **Athens**.


---